# Jog mode adatelemzés

In [296]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
anomaly_num = 0

In [297]:
import influxdb

In [298]:
df = pd.read_csv("PLC_NBH170.csv", sep=";")

In [299]:
def switch(i):
    switcher = {
        'KBBUD10449-NBH170-PLCStationThing': "10449",
        'KBBUD10704-NBH170-PLCStationThing': "10704",
        'KBBUD10469-NBH170-PLCStationThing': "10469",
        'KBBUD10460-NBH170-PLCStationThing': "10460",
        'KBBUD10395-NBH170-PLCStationThing': "10395",
        'KBBUD10424-NBH170-PLCStationThing': "10424",
        'KBBUD10419-NBH170-PLCStationThing': "10419",
        'KBBUD10396-NBH170-PLCStationThing': "10396"
        
    }
    return switcher.get(i,"invalid")

In [300]:
df['Machine'] = df['Machine'].apply(lambda x: switch(x))
df['StatusTime'] = df['StatusTime'].apply(lambda x: x[:-4])

del df['StatusType']
del df['description']
del df['CreationTime']

# JOG/Auto MODE

In [301]:
dfj = df[df['Status']=='JOG']
dfm = df[df['Status']=='MODE']

In [302]:
df = pd.concat([dfj,dfm])

In [303]:
del df['Status']
df = df.sort_values(by='StatusTime')

In [304]:
list = []
list.append(df[df['Machine']=='10449'])
list.append(df[df['Machine']=='10704'])
list.append(df[df['Machine']=='10469'])
list.append(df[df['Machine']=='10460'])
list.append(df[df['Machine']=='10395'])
list.append(df[df['Machine']=='10424'])
list.append(df[df['Machine']=='10419'])
list.append(df[df['Machine']=='10396'])

In [305]:
df

,Id,Machine,StatusTime,SubStatus
74,227,10395,2019-08-21 13:29:05,ON
83,255,10395,2019-08-21 13:30:04,AUTO
85,267,10395,2019-08-21 13:30:24,ON
90,274,10395,2019-08-21 13:31:43,AUTO
139,522,10396,2019-08-21 13:47:25,ON
...,...,...,...,...
614724,2012220,10704,2019-11-19 12:35:53,ON
614743,2012242,10395,2019-11-19 12:37:57,AUTO
614785,2012296,10704,2019-11-19 12:41:54,ON
614886,2012499,10395,2019-11-19 12:50:59,ON


In [306]:
# Anomália kezelés 1: több START jön egymás után, akkor az ON érvényes, több AUTO jön egymás után, az utolsó az érvényes

def anomaly_handle_1(list):
    global anomaly_num
    anomaly_num = 1

    for j in range(len(list)):
        for i in range(len(list[j])-1):
            if(list[j]['SubStatus'].iloc[i] == 'ON' ):
                if(list[j]['SubStatus'].iloc[i+1] == 'AUTO'):     
                    list[j]['SubStatus'].iloc[i+1] = 'Anomaly'
            elif(list[j]['SubStatus'].iloc[i] == 'Anomaly'):
                if(list[j]['SubStatus'].iloc[i+1] == 'ON'):     
                    list[j]['SubStatus'].iloc[i+1] = 'Anomaly'
    for j in range(len(list)):
        for i in range(len(list[j])-1):
            if(list[j]['SubStatus'].iloc[i] == 'ON' ):
                if(list[j]['SubStatus'].iloc[i+1] == 'ON'):     
                    list[j]['SubStatus'].iloc[i] = 'Anomaly'

In [307]:
# Anomália kezelés 2: több ON jön egymás után, akkor az UTOLSÓ érvényes, több AUTO jön egymás után, az utolsó az érvényes

def anomaly_handle_2(list):
    global anomaly_num
    anomaly_num = 2

    for j in range(len(list)):
        for i in range(len(list[j])-1):
            if(list[j]['SubStatus'].iloc[i] == 'ON' ):
                if(list[j]['SubStatus'].iloc[i+1] == 'ON'):     
                    list[j]['SubStatus'].iloc[i] = 'Anomaly'
    for j in range(len(list)):
        for i in range(len(list[j])-1):
            if(list[j]['SubStatus'].iloc[i] == 'AUTO' ):
                if(list[j]['SubStatus'].iloc[i+1] == 'AUTO'):     
                    list[j]['SubStatus'].iloc[i] = 'Anomaly'

In [308]:
anomaly_handle_2(list)

In [309]:
j = 0
k = 0
list_num = 7
for i in range(len(list[list_num])):
    if(list[list_num]['SubStatus'].iloc[i] == 'ON'):
        j = j + 1
    elif(list[list_num]['SubStatus'].iloc[i] == 'AUTO'):
        k = k + 1

print("ON: " + str(j))
print("AUTO: " + str(k))

ON: 785
AUTO: 785


In [310]:
list_anomaly = []

for i in range(len(list)): 
    list_anomaly.insert(i, list[i]) 

In [311]:
for j in range(len(list)):
    list[j] = list[j][list[j]['SubStatus']!='Anomaly']

# idő hozzárendelése gépenként

In [312]:
# on kell hogy kezdődjön és auto-val végződnie

In [313]:
for i in range(len(list_anomaly)):
    if(list_anomaly[i].iloc[0]['SubStatus'] != 'ON'):
        list_anomaly[i] = list_anomaly[i].drop([list_anomaly[i].index[0]])
    if(list_anomaly[i].iloc[len(list_anomaly[i]) - 1]['SubStatus'] != 'AUTO'):
        list_anomaly[i] = list_anomaly[i].drop([list_anomaly[i].index[len(list_anomaly[i])-1]])
    if(list_anomaly[i].iloc[len(list_anomaly[i]) - 1]['SubStatus'] == 'Anomaly'):
        list_anomaly[i] = list_anomaly[i].drop([list_anomaly[i].index[len(list_anomaly[i])-1]])

for i in range(len(list)):
    if(list[i].iloc[0]['SubStatus'] != 'ON'):
        list[i] = list[i].drop([list[i].index[0]])
    if(list[i].iloc[len(list[i]) - 1]['SubStatus'] != 'AUTO'):
        list[i] = list[i].drop([list[i].index[len(list[i])-1]])

In [314]:
list_anomaly[1] = list_anomaly[1].drop([614601,613226,613219,614681])

In [315]:
from datetime import datetime, timedelta

In [316]:
def timeInManualyWorking(list): 
    for j in range(len(list)):    
        list[j]['timeInManualyWorking'] = timedelta(0)
        for i in range(len(list[j])-1)[0::2]:
            timeACTIVE_Str = list[j]['StatusTime'].iloc[i]
            timeEND_Str = list[j]['StatusTime'].iloc[i+1]
            timeStart = datetime.strptime(timeACTIVE_Str,'%Y-%m-%d %H:%M:%S')
            timeEND = datetime.strptime(timeEND_Str,'%Y-%m-%d %H:%M:%S')
            list[j]['timeInManualyWorking'].iloc[i+1] = timeEND-timeStart

In [317]:
def timeInAutoWorking(list): 
    for j in range(len(list)):
        list[j]['timeInAutoWorking'] = timedelta(0)
        for i in range(len(list[j])-1)[1::2]:
            timeACTIVE_Str = list[j]['StatusTime'].iloc[i]
            timeEND_Str = list[j]['StatusTime'].iloc[i+1]
            timeStart = datetime.strptime(timeACTIVE_Str,'%Y-%m-%d %H:%M:%S')
            timeEND = datetime.strptime(timeEND_Str,'%Y-%m-%d %H:%M:%S')
            list[j]['timeInAutoWorking'].iloc[i+1] = timeEND-timeStart

In [318]:
def timeInAnomaly(list):
    for j in range(len(list)):
        list[j]['timeInAnomaly'] = timedelta(0)

        i = 0
        while(i <= len(list[j])-1):
            if(list[j]['SubStatus'].iloc[i] == 'Anomaly' ):
                anomalyStart_Str = list[j]['StatusTime'].iloc[i]
                #ameddig anomália jön, addig megyünk
                k = 1
                while list[j]['SubStatus'].iloc[k + i] == 'Anomaly':
                    k = k + 1

                if(k == 1):
                    i += 1
                    pass
                else:
                    anomalyEND_Str = list[j]['StatusTime'].iloc[i + k]
                    timeStart = datetime.strptime(anomalyStart_Str,'%Y-%m-%d %H:%M:%S')
                    timeEND = datetime.strptime(anomalyEND_Str,'%Y-%m-%d %H:%M:%S')
                    list[j]['timeInAnomaly'].iloc[i + k] = timeEND-timeStart  
                    i = i + k
                    k = 1
            else:
                i += 1 

In [319]:
timeInManualyWorking(list)
timeInAutoWorking(list)

In [320]:
timeInAnomaly(list_anomaly)

In [321]:
print('manually working time','','autoamata working time',' ','all time sum of the machine')
print('')
for i in range(len(list)):
    print(list[i]['timeInManualyWorking'].sum(),'     ', list[i]['timeInAutoWorking'].sum(),'       ',list[i]['timeInManualyWorking'].sum()+list[i]['timeInAutoWorking'].sum())

manually working time  autoamata working time   all time sum of the machine

11 days 16:07:38       77 days 23:12:24         89 days 15:20:02
24 days 16:54:50       64 days 22:49:43         89 days 15:44:33
44 days 05:40:56       45 days 11:24:11         89 days 17:05:07
18 days 01:53:36       35 days 14:24:00         53 days 16:17:36
34 days 08:55:07       55 days 14:29:29         89 days 23:24:36
26 days 17:06:29       62 days 22:36:00         89 days 15:42:29
25 days 05:20:32       64 days 08:55:29         89 days 14:16:01
6 days 07:57:08       83 days 14:41:03         89 days 22:38:11


In [322]:
for i in range(len(list_anomaly)):
    print(str(i+1) + " számú gép: ",list_anomaly[i]['timeInAnomaly'].sum())

1 számú gép:  0 days 20:43:11
2 számú gép:  16 days 01:59:34
3 számú gép:  3 days 16:56:11
4 számú gép:  3 days 23:12:45
5 számú gép:  14 days 07:11:31
6 számú gép:  13 days 14:21:19
7 számú gép:  11 days 16:05:54
8 számú gép:  0 days 03:23:21


In [323]:
for i in range(len(list_anomaly)-1):
    del list_anomaly[i]['Machine']
    del list_anomaly[i]['StatusTime']
    del list_anomaly[i]['SubStatus']

In [324]:
list_new = []
for i in range(len(list)-1):
    list_new.append(pd.merge(list[i], list_anomaly[i], left_on='Id', right_on='Id', how='left'))

In [325]:
df = pd.concat(list_new)

In [326]:
df

,Id,Machine,StatusTime,SubStatus,timeInManualyWorking,timeInAutoWorking,timeInAnomaly
0,4671,10449,2019-08-21 17:39:56,ON,00:00:00,00:00:00,0 days
1,4902,10449,2019-08-21 17:55:22,AUTO,00:15:26,00:00:00,0 days
2,21373,10449,2019-08-22 10:05:40,ON,00:00:00,16:10:18,0 days
3,21381,10449,2019-08-22 10:05:59,AUTO,00:00:19,00:00:00,0 days
4,23575,10449,2019-08-22 12:04:12,ON,00:00:00,01:58:13,0 days
...,...,...,...,...,...,...,...
1371,1998958,10419,2019-11-19 00:45:19,AUTO,00:05:21,00:00:00,0 days
1372,2003879,10419,2019-11-19 04:00:30,ON,00:00:00,03:15:11,0 days
1373,2004218,10419,2019-11-19 04:11:13,AUTO,00:10:43,00:00:00,0 days
1374,2004844,10419,2019-11-19 04:38:04,ON,00:00:00,00:26:51,0 days


In [327]:
import os
from datetime import datetime as dt
import datetime
import pytz, time
import gc

os.environ['TZ'] = 'Europe/Budpest'
CET = pytz.timezone('Europe/Budapest')

In [328]:
df

,Id,Machine,StatusTime,SubStatus,timeInManualyWorking,timeInAutoWorking,timeInAnomaly
0,4671,10449,2019-08-21 17:39:56,ON,00:00:00,00:00:00,0 days
1,4902,10449,2019-08-21 17:55:22,AUTO,00:15:26,00:00:00,0 days
2,21373,10449,2019-08-22 10:05:40,ON,00:00:00,16:10:18,0 days
3,21381,10449,2019-08-22 10:05:59,AUTO,00:00:19,00:00:00,0 days
4,23575,10449,2019-08-22 12:04:12,ON,00:00:00,01:58:13,0 days
...,...,...,...,...,...,...,...
1371,1998958,10419,2019-11-19 00:45:19,AUTO,00:05:21,00:00:00,0 days
1372,2003879,10419,2019-11-19 04:00:30,ON,00:00:00,03:15:11,0 days
1373,2004218,10419,2019-11-19 04:11:13,AUTO,00:10:43,00:00:00,0 days
1374,2004844,10419,2019-11-19 04:38:04,ON,00:00:00,00:26:51,0 days


In [329]:
#df = df[['Id', 'Machine', 'StatusTime', 'Status', 'SubStatus']]
df = df[['Id', 'Machine', 'StatusTime', 'SubStatus', 'timeInManualyWorking', 'timeInAutoWorking', 'timeInAnomaly']]
df = df.astype({'StatusTime' : 'datetime64'})
df.index = df.StatusTime
df.index = df.index.tz_localize('Europe/Budapest')
df = df.drop(columns='StatusTime')
df.sort_values(inplace=True, axis='index', by='StatusTime')
df.head()

,Id,Machine,SubStatus,timeInManualyWorking,timeInAutoWorking,timeInAnomaly
StatusTime,,,,,,
2019-08-21 13:29:05+02:00,227,10395,ON,00:00:00,00:00:00,0 days
2019-08-21 13:30:04+02:00,255,10395,AUTO,00:00:59,00:00:00,0 days
2019-08-21 13:30:24+02:00,267,10395,ON,00:00:00,00:00:20,0 days
2019-08-21 13:31:43+02:00,274,10395,AUTO,00:01:19,00:00:00,0 days
2019-08-21 13:56:16+02:00,632,10460,ON,00:00:00,00:00:00,0 days


In [330]:
df['ts_auto'] = 0
for i in range(len(df)):
    df['ts_auto'].iloc[i] = df['timeInAutoWorking'].iloc[i].total_seconds() / 3600

In [331]:
df['ts_man'] = 0
for i in range(len(df)):
    df['ts_man'].iloc[i] = df['timeInManualyWorking'].iloc[i].total_seconds() / 3600

In [332]:
df['ts_anom'] = 0
for i in range(len(df)):
    df['ts_anom'].iloc[i] = df['timeInAnomaly'].iloc[i].total_seconds() / 3600

In [333]:
del df['timeInManualyWorking']
del df['timeInAutoWorking']
del df['timeInAnomaly']
df

,Id,Machine,SubStatus,ts_auto,ts_man,ts_anom
StatusTime,,,,,,
2019-08-21 13:29:05+02:00,227,10395,ON,0.000000,0.000000,0.0
2019-08-21 13:30:04+02:00,255,10395,AUTO,0.000000,0.016389,0.0
2019-08-21 13:30:24+02:00,267,10395,ON,0.005556,0.000000,0.0
2019-08-21 13:31:43+02:00,274,10395,AUTO,0.000000,0.021944,0.0
2019-08-21 13:56:16+02:00,632,10460,ON,0.000000,0.000000,0.0
...,...,...,...,...,...,...
2019-11-19 12:27:55+01:00,2012148,10395,AUTO,0.000000,0.044167,0.0
2019-11-19 12:34:56+01:00,2012204,10395,ON,0.116944,0.000000,0.0
2019-11-19 12:37:57+01:00,2012242,10395,AUTO,0.000000,0.050278,0.0


df.to_csv('nc_status_2.csv', sep=";")

In [334]:
import influxdb

# The data export can be significantly accelerated by appropriate batch_size

client = influxdb.DataFrameClient(host='152.66.34.82', port=8086)
client.switch_database('knorr')
client.query(query='DROP MEASUREMENT "jog_mode"', database='knorr')
client.write_points(df, database='knorr', measurement='jog_mode', tag_columns=['Machine', 'SubStatus'], field_columns=['ts_auto','ts_man','ts_anom'], time_precision='ms', batch_size=10000, protocol='line')
client.query(query='SELECT * FROM jog_mode', database='knorr')

defaultdict(list,
            {'jog_mode':                           Machine SubStatus  ts_anom   ts_auto    ts_man
             2019-08-21 11:29:05+00:00   10395        ON      0.0  0.000000  0.000000
             2019-08-21 11:30:04+00:00   10395      AUTO      0.0  0.000000  0.016389
             2019-08-21 11:30:24+00:00   10395        ON      0.0  0.005556  0.000000
             2019-08-21 11:31:43+00:00   10395      AUTO      0.0  0.000000  0.021944
             2019-08-21 11:56:16+00:00   10460        ON      0.0  0.000000  0.000000
             ...                           ...       ...      ...       ...       ...
             2019-11-19 11:27:55+00:00   10395      AUTO      0.0  0.000000  0.044167
             2019-11-19 11:34:56+00:00   10395        ON      0.0  0.116944  0.000000
             2019-11-19 11:37:57+00:00   10395      AUTO      0.0  0.000000  0.050278
             2019-11-19 11:50:59+00:00   10395        ON      0.0  0.217222  0.000000
             2019-11-19 